В домашньому завданні до даного модулю ви потренуєтесь робити тестове завдання для влаштування на роботу. За даними акселерометра з мобільного телефону потрібно класифікувати, якою діяльністю займається людина: йде, стоїть, біжить чи йде по сходах. Знайти датасет ви можете за посиланням.



Використайте алгоритми SVM та випадковий ліс з бібліотеки scikit-learn. Як характеристики можете брати показники з акселерометра, проте щоб покращити результати роботи алгоритмів, спочатку можна підготувати наш датасет і розрахувати часові ознаки (time domain features). Більше ці характеристики описані в даній статті.



Порівняйте результати роботи обох алгоритмів на різних фічах та різні моделі між собою.



Порівняйте результати роботи обох алгоритмів на різних фічах та різні моделі між собою. Використайте метод classification report для порівняння.



Порівняння моделей на основі однієї метрики(такої як Accuracy)- не приймається.

In [1]:
import os
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
import pandas as pd
import os
import zipfile
import re

data_folder = 'homework.zip'
all_data = pd.DataFrame()

with zipfile.ZipFile(data_folder, 'r') as z:
    # List all files and directories in the ZIP
    for file_info in z.infolist():
        file_name = file_info.filename
        # Check if it's a file (not a directory) and ends with .csv
        if not file_info.is_dir() and file_name.endswith('.csv'):
            # The directory name in the archive acts as the 'activity'
            activity_folder = os.path.dirname(file_name)
            # Extracting numeric identifier from the file name if needed
            # Assuming the naming convention is '[activity_folder]/[activity]-[number].csv'
            num_match = re.search(r'-(\d+)\.csv', file_name)
            num = int(num_match.group(1)) if num_match else None

            with z.open(file_name) as f:
                activity_data = pd.read_csv(f)
                activity_data['activity'] = activity_folder.split('/')[-1]  # Getting the last part after '/'
                activity_data['num'] = num
                all_data = pd.concat([all_data, activity_data], ignore_index=True)

all_data

,accelerometer_X,accelerometer_Y,accelerometer_Z,activity,num
0,1.000776,4.616021,8.576031,idle,1
1,0.718261,4.209007,8.446744,idle,1
2,-0.909797,-0.282516,9.203311,idle,1
3,5.099650,0.148441,8.418014,idle,1
4,1.762132,-0.162806,9.251195,idle,1
...,...,...,...,...,...
193855,5.109226,-15.452178,-1.470040,walking,999
193856,6.373365,-11.741165,-8.226476,walking,999
193857,3.289633,-9.993398,-0.383072,walking,999
193858,-2.978387,-3.050213,1.273715,walking,999


In [3]:
all_data

,accelerometer_X,accelerometer_Y,accelerometer_Z,activity,num
0,1.000776,4.616021,8.576031,idle,1
1,0.718261,4.209007,8.446744,idle,1
2,-0.909797,-0.282516,9.203311,idle,1
3,5.099650,0.148441,8.418014,idle,1
4,1.762132,-0.162806,9.251195,idle,1
...,...,...,...,...,...
193855,5.109226,-15.452178,-1.470040,walking,999
193856,6.373365,-11.741165,-8.226476,walking,999
193857,3.289633,-9.993398,-0.383072,walking,999
193858,-2.978387,-3.050213,1.273715,walking,999


In [4]:
df=all_data.drop('num',axis=1)

In [5]:
df

,accelerometer_X,accelerometer_Y,accelerometer_Z,activity
0,1.000776,4.616021,8.576031,idle
1,0.718261,4.209007,8.446744,idle
2,-0.909797,-0.282516,9.203311,idle
3,5.099650,0.148441,8.418014,idle
4,1.762132,-0.162806,9.251195,idle
...,...,...,...,...
193855,5.109226,-15.452178,-1.470040,walking
193856,6.373365,-11.741165,-8.226476,walking
193857,3.289633,-9.993398,-0.383072,walking
193858,-2.978387,-3.050213,1.273715,walking


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

X = df.drop('activity', axis=1)
y = df['activity']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


svm_classifier = SVC(kernel='linear', C=1.0)
svm_classifier.fit(X_train, y_train)

svm_predictions = svm_classifier.predict(X_test)

svm_report = classification_report(y_test, svm_predictions, target_names=['idle', 'running', 'stairs', 'walking'])


print("SVM Classification Report:\n", svm_report)


d:\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


SVM Classification Report:
               precision    recall  f1-score   support

        idle       0.81      0.78      0.80      6203
     running       0.86      0.85      0.85     20485
      stairs       0.00      0.00      0.00      1022
     walking       0.76      0.86      0.81     11062

    accuracy                           0.82     38772
   macro avg       0.61      0.62      0.62     38772
weighted avg       0.80      0.82      0.81     38772



d:\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [7]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)
rf_predictions = rf_classifier.predict(X_test)
rf_report = classification_report(y_test, rf_predictions, target_names=['idle', 'running', 'stairs', 'walking'])

print("Random Forest Classification Report:\n", rf_report)

Random Forest Classification Report:
               precision    recall  f1-score   support

        idle       1.00      1.00      1.00      6203
     running       1.00      1.00      1.00     20485
      stairs       1.00      0.99      0.99      1022
     walking       1.00      1.00      1.00     11062

    accuracy                           1.00     38772
   macro avg       1.00      1.00      1.00     38772
weighted avg       1.00      1.00      1.00     38772

